# Create Data Structure

Using the data model json, create a structure as file geodatabase

#### Import statements

In [1]:
# import the arcpy libary
import arcpy
arcpy.env.overwriteOutput = True

# import other modules
import json
import copy
import os

# set default spatial reference
sr = arcpy.SpatialReference(r'C:\Users\rjc.EAGLE\AppData\Local\ESRI\ArcGISPro\Favorites\NZGD 2000 New Zealand Transverse Mercator.prj')

#### Set output path

In [2]:
basePath = r'C:\Users\rjc.EAGLE\Documents\GitHub\agriculture-data-model'
outputFolders = os.listdir('{0}\\output'.format(basePath))

n = 0
for folder in outputFolders:
    print('{0} - {1}'.format(n, folder))
    n+=1
    
selectedFolder = int(input('Select a output folder: '))
outputPath = '{0}\\output\\{1}'.format(basePath, outputFolders[selectedFolder])
print('Output path: {0}'.format(outputPath))

0 - example-farm
1 - miraka
2 - test
Select a output folder: 1
Output path: C:\Users\rjc.EAGLE\Documents\GitHub\agriculture-data-model\output\miraka


#### Set paths and load in Data Model

In [13]:
data = {}
try:    
    data.update(json.load(open('{0}\\service\\{1}.json'.format(outputPath, 'datamodel'))))
    data.update(json.load(open('{0}\\service\\{1}.json'.format(outputPath, 'tables'))))
    data.update(json.load(open('{0}\\service\\{1}.json'.format(outputPath, 'domains'))))
except Exception as e:
    print (e)
print(data)

{'services': [{'name': 'administration', 'alias': 'Administration', 'tables': ['activities', 'additional_details', 'measurements', 'staff', 'logs'], 'layers': [{'name': 'access_points', 'alias': 'Access Points', 'type': 'POINT', 'children': ['activities'], 'fields': [{'name': 'name', 'alias': 'Address', 'type': 'TEXT', 'length': 255, 'domain': None}, {'name': 'alias', 'alias': 'Alias', 'type': 'TEXT', 'length': 255, 'domain': None}, {'name': 'unique_number', 'alias': 'Unique Number', 'type': 'SHORT', 'length': None, 'domain': None}, {'name': 'status', 'alias': 'Status', 'type': 'TEXT', 'length': 255, 'domain': 'status'}, {'name': 'start_date', 'alias': 'Start Date', 'type': 'DATE', 'length': None, 'domain': None}, {'name': 'end_date', 'alias': 'End Date', 'type': 'DATE', 'length': None, 'domain': None}, {'name': 'class', 'alias': 'Class', 'type': 'TEXT', 'length': 255, 'domain': 'primary_secondary'}, {'name': 'notes', 'alias': 'Notes', 'type': 'TEXT', 'length': 1000, 'domain': None}], 

#### Define functions

In [4]:
def createItems(workspace, item):   
    if item['type'] == 'TABLE':        
        result = arcpy.management.CreateTable(workspace, item['name'], out_alias=item['alias'])        
    else:
        result = arcpy.management.CreateFeatureclass(workspace, item['name'], item['type'], spatial_reference=sr, out_alias=item['alias'])
    itemFields = item['fields']
    if itemFields != None:
        fields = itemFields
    else:
        table = [x for x in data['tables'] if item['master'] == x['name']][0]
        fields = table['fields']
    for field in fields:    
        arcpy.management.AddField(result, field['name'], field['type'], field_length=field['length'], field_alias=field['alias'], field_domain=field['domain'])
        if field['name'] == 'status':
            arcpy.management.AssignDefaultToField(result, field['name'], "Current")
        
    arcpy.management.AddGlobalIDs(result)
    print("    + {0}".format(item['name']))

def createRelationships(workspace, item):  
    origin_table = '{0}\\{1}'.format(workspace, item['name'])     
    parent_id = 'parent_{0}_id'.format(item['name'])
    parent_alias = 'Parent {0} ID'.format(item['name'].replace('_', ' '))
    children = item['children']
    for child in children:
        child_path = '{0}\\{1}'.format(workspace, child)       
        relationship_class = '{0}_{1}'.format(item['name'], child)
        arcpy.management.AddField(child_path, parent_id, 'GUID') 
        arcpy.management.CreateRelationshipClass(origin_table, child_path, relationship_class, 'Simple', child, item['name'], None, 'ONE_TO_MANY', None, 'GlobalID', parent_id)  
    print("    + {0}".format(item['name']))
        
def enableEditorTracking(workspace, item): 
    in_table = '{0}\\{1}'.format(workspace, item['name'])
    arcpy.management.AddField(in_table, field_name='creator', field_type='TEXT', field_alias='Creator')
    arcpy.management.AddField(in_table, field_name='creation_date', field_type='DATE', field_alias='Creation Date')
    arcpy.management.AddField(in_table, field_name='last_editor', field_type='TEXT', field_alias='Last Editor')
    arcpy.management.AddField(in_table, field_name='last_edit_date', field_type='DATE', field_alias='Last Edit Date')
    arcpy.management.EnableEditorTracking(in_table, 'creator', 'creation_date', 'last_editor', 'last_edit_date') 
    arcpy.management.EnableAttachments(in_table)
    print("    + {0}".format(item['name']))

#### Create data structure

In [14]:
outputDataPath = '{0}\\data'.format(outputPath)
arcpy.env.workspace = outputDataPath
existingWorkspaces = arcpy.ListWorkspaces("*", "FileGDB")

for service in data['services'][12:13]:
    
    print(service['name'])
    
    # creat the workspace, this will overwrite the existing workspaces
    workspace = arcpy.CreateFileGDB_management(outputDataPath, "{0}.gdb".format(service['name'])).getOutput(0)
    arcpy.env.workspace = workspace
    
    print("  - creating domains")
    #existing_domains = [x.name for x in arcpy.da.ListDomains(workspace)]  
    domains = [x for x in data['domains'] if service['name'] in x['workspace'] or '*' == x['workspace'][0]]
    for domain in domains:
        arcpy.management.CreateDomain(workspace, domain['name'], domain['description'], domain['field_type'], domain['domain_type'])
        for code in domain['codes']:
            arcpy.AddCodedValueToDomain_management(workspace, domain['name'], code, domain['codes'][code])
    print("    + completed")
    
    # create the tables
    print("  - creating tables")
    tables = [x for x in data['tables'] if x['name'] in service['tables']]
    for table in tables:        
        createItems(workspace, table)
           
    # create items
    print("  - creating items")
    layers = service['layers']
    for item in layers:
        createItems(workspace, item) 
       
    # create relationships
    print("  - creating relationships")
    for item in layers:        
        createRelationships(workspace, item)  
    for table in tables:        
        createRelationships(workspace, table)  
        
    # enable editor tracking
    print("  - enabling editor tracking")
    for item in layers:        
        enableEditorTracking(workspace, item)  
        
    print(' ')


health_and_safety
  - creating domains
    + completed
  - creating tables
    + hazards
    + logs
    + injuries
  - creating items
    + incidents
    + point_hazards
    + line_hazards
    + polygon_hazards
  - creating relationships
    + incidents
    + point_hazards
    + line_hazards
    + polygon_hazards
    + hazards
    + logs
    + injuries
  - enabling editor tracking
    + incidents
    + point_hazards
    + line_hazards
    + polygon_hazards
 


In [11]:
data['services'][12:13]

[{'name': 'health_and_safety',
  'alias': 'Health and Safety',
  'tables': ['hazards', 'injuries', 'logs'],
  'layers': [{'name': 'incidents',
    'alias': 'Incidents',
    'type': 'POINT',
    'children': ['injuries'],
    'fields': [{'name': 'name',
      'alias': 'Name',
      'type': 'TEXT',
      'length': 255,
      'domain': None},
     {'name': 'alias',
      'alias': 'Alias',
      'type': 'TEXT',
      'length': 255,
      'domain': None},
     {'name': 'unique_number',
      'alias': 'Unique Number',
      'type': 'SHORT',
      'length': None,
      'domain': None},
     {'name': 'incident_date',
      'alias': 'Date of Incident',
      'type': 'DATE',
      'length': None,
      'domain': None},
     {'name': 'report_date',
      'alias': 'Date of Report',
      'type': 'DATE',
      'length': None,
      'domain': None},
     {'name': 'class',
      'alias': 'Class',
      'type': 'TEXT',
      'length': 255,
      'domain': 'incident_class'},
     {'name': 'description',

In [ ]:
data['services'][-2:]